<a href="https://colab.research.google.com/github/mati-avila/FIA2023-TPI/blob/main/FIA2023_TPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ALUMNO: AVILA, JORGE MATIAS - LU1738 - LICENCIATURA EN SISTEMAS

In [ ]:
install.packages('lubridate')#para trabajar con fechas
install.packages('ggplot2')#Para graficos
install.packages('ggdist')#Para graficos (histogramas)

In [ ]:
install.packages("e1071")#svm
install.packages("randomForest") #randomForest
install.packages("clock") #necesaria para instalar 'caret'
install.packages("fastDummies") #para realizar el casteo (one-hot encoding)

In [ ]:
install.packages("caret") #para el particionado del df

In [ ]:
library(lubridate)
library(ggplot2)
library(ggdist)
library(dplyr)

In [8]:
# Los .csv deben estar cargados en el entorno, click en los tres puntos y copiamos la ruta de acceso
df_princess <- read.csv("/content/princess.csv")
df_wine <- read.csv("/content/wine.csv")

In [ ]:
head(df_princess) # muestra las primeras filas del dataset, si queremos ver las últimas filas usamos tail()

In [ ]:
head(df_wine) # muestra las primeras filas del dataset, si queremos ver las últimas filas usamos tail()

**Seleccion**

Seleccione el conjunto de datos sobre vinos debido a su relevancia e interés para el análisis. Podemos decir que este conjunto consta de 1599 observaciones distribuidas en 13 variables y ofrece una variedad de aspectos de los vinos, lo que permite explorar relaciones y patrones interesantes.
La calidad del vino la hemos clasificado en dos categorías: 'quality_bad' (calidad mala) y 'quality_good' (calidad buena), proporcionando una dimensión adicional para el análisis.

# LIMPIEZA

In [ ]:
# Vemos si hay campos nulos en alguna columna
colSums(is.na(df_wine))

Como el data set esta limpio, y esta baleanceado. Por lo tanto se comienza a entrenar.

Pero una propuesta para baleancear los datos, seria duplicar aleatoriamente instancias de la clase en el conjunto de datos. Esto se logra mediante la replicación de las filas correspondientes a esa clase.

In [ ]:
#importamos la libreria
library(e1071)
library(randomForest)
library(dplyr)
library(caret)
library(reshape2)
library(fastDummies)

# SVM

In [13]:
columnas_a_castear <- c('quality')
# Utilizamos fastDummies para convertir las columnas en filas
df_wineSVM <- fastDummies::dummy_cols(df_wine, select_columns = columnas_a_castear)

In [88]:
str(df_wineSVM)

'data.frame':	1599 obs. of  13 variables:
 $ fixed.acidity       : num  7.4 7.8 7.8 11.2 7.4 7.4 7.9 7.3 7.8 7.5 ...
 $ volatile.acidity    : num  0.7 0.88 0.76 0.28 0.7 0.66 0.6 0.65 0.58 0.5 ...
 $ citric.acid         : num  0 0 0.04 0.56 0 0 0.06 0 0.02 0.36 ...
 $ residual.sugar      : num  1.9 2.6 2.3 1.9 1.9 1.8 1.6 1.2 2 6.1 ...
 $ chlorides           : num  0.076 0.098 0.092 0.075 0.076 0.075 0.069 0.065 0.073 0.071 ...
 $ free.sulfur.dioxide : num  11 25 15 17 11 13 15 15 9 17 ...
 $ total.sulfur.dioxide: num  34 67 54 60 34 40 59 21 18 102 ...
 $ density             : num  0.998 0.997 0.997 0.998 0.998 ...
 $ pH                  : num  3.51 3.2 3.26 3.16 3.51 3.51 3.3 3.39 3.36 3.35 ...
 $ sulphates           : num  0.56 0.68 0.65 0.58 0.56 0.56 0.46 0.47 0.57 0.8 ...
 $ alcohol             : num  9.4 9.8 9.8 9.8 9.4 9.4 9.4 10 9.5 10.5 ...
 $ quality_bad         : int  1 1 1 0 1 1 1 0 0 1 ...
 $ quality_good        : Factor w/ 2 levels "0","1": 1 1 1 2 1 1 1 2 2 1 ...


In [15]:
df_wineSVM$quality_good <- factor(df_wineSVM$quality_good)

In [16]:
# Eliminamos las columnas en df_credit_cardSVM
df_wineSVM <- df_wineSVM[, !(names(df_wineSVM) %in% columnas_a_castear)]

In [ ]:
table(df_wineSVM$quality_good)

In [65]:
set.seed(2023) #seteamos la semilla para que el experimento sea reproducible
t.ids <- createDataPartition(df_wineSVM$quality_good, p =0.7, list=F) #particionamos el df en 70% para entrenamiento y 30% para prueba
modelo <- svm(quality_good ~ ., data = df_wineSVM[t.ids, ]) #entrenamiento
prediccion <- predict(modelo, df_wineSVM[-t.ids, ]) #prueba
matriz_confusion <- table(df_wineSVM$quality_good[-t.ids], prediccion, dnn = c('Real', 'Predicho')) #creamos una matriz de confusión

In [ ]:
print(matriz_confusion)

In [ ]:
sum(prediccion != df_wineSVM$quality_good[-t.ids])/length(df_wineSVM$quality_good[-t.ids])

In [ ]:
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion)

In [ ]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

# RANDONFOREST

In [74]:
set.seed(2023)
t.ids <- createDataPartition(df_wineSVM$quality_good, p =0.7, list=F)
modelo_rf <- randomForest(quality_good ~ ., data = df_wineSVM[t.ids, ])
prediccion_rf <- predict(modelo_rf, newdata = df_wineSVM[-t.ids, ])
matriz_confusion <- table(prediccion_rf, df_wineSVM$quality_good[-t.ids], dnn = c('Real' ,'Predicho'))

In [ ]:
print(matriz_confusion)

In [ ]:
1 - sum(diag(matriz_confusion)) / sum(matriz_confusion)

In [ ]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

# Redes Neuronales

In [ ]:
install.packages('neuralnet')#para trabajar con redes neuronales

In [ ]:
# Instala y carga el paquete neuralnet si aún no lo has hecho
# install.packages("neuralnet")
library(neuralnet)

In [85]:
# Configuración de la semilla para reproducibilidad
set.seed(2023)

# Partición de datos
t.ids <- createDataPartition(df_wineSVM$quality_good, p = 0.7, list = FALSE)

# Creación del objeto formula
formula <- as.formula("quality_good ~ .")

# Creación del modelo de red neuronal
modelo <- neuralnet(
  formula,
  data = df_wineSVM[t.ids, ],
  hidden = c(5, 2), # Número de neuronas en las capas ocultas
  linear.output = TRUE # Utiliza salida lineal
)

# Predicciones
prediccion <- predict(modelo, newdata = df_wineSVM[-t.ids, ])
prediccion_binaria <- ifelse(prediccion[, 1] > 1, 0, 1)

# Matriz de confusión
matriz_confusion <- table(
  Real = df_wineSVM$quality_good[-t.ids],
  Predicho = prediccion_binaria
)



In [ ]:
print(matriz_confusion)

In [87]:
# Precisión
precision <- (matriz_confusion[1, 1] + matriz_confusion[2, 2]) / sum(matriz_confusion)

# Sensibilidad (Recall)
sensibilidad <- matriz_confusion[2, 2] / sum(matriz_confusion[2, ])

print(precision)
print(sensibilidad)

[1] 0.9185804
[1] 1


**Los Resultados**
La precision del SVM es de 99%, la de Randon forest es de casi 100% y en cambio la precision de Red Neuronal es de 91%.

**Conclusiones**
SVM está mostrando un rendimiento excepcionalmente alto en términos de precisión, pero habria que evaluar si es por un sobreajuste.

Random Forest al igual que con SVM, la precisión cercana al 100% en Random Forest sugiere un rendimiento excepcional. Sin embargo, al igual que antes, verifica si hay sobreajuste.

Red Neuronal aunque esta precisión de la red neuronal es ligeramente inferior, es importante destacar que podría estar generalizando mejor a nuevos datos y poder aprender patrones mas complejos.